In [1]:
import torch
import torch.optim as optim
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
import os
import random
import pandas as pd
from torchvision.transforms import RandomRotation, RandomHorizontalFlip, RandomVerticalFlip, RandomResizedCrop
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:


class GAN(nn.Module):
    def __init__(self, image_size=128, latent_dim=100):
        super(GAN, self).__init__()
        
        self.image_size = image_size
        self.latent_dim = latent_dim

        # Generator
        self.generator = nn.Sequential(
            nn.Linear(latent_dim, 512 * (image_size // 16) * (image_size // 16)),
            nn.BatchNorm1d(512 * (image_size // 16) * (image_size // 16)),
            nn.ReLU(inplace=True),
            nn.Unflatten(1, (512, image_size // 16, image_size // 16)),
            
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()  # Output in range [-1, 1]
        )

        # Discriminator
        self.discriminator = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Flatten(),
            nn.Linear(512 * (image_size // 16) * (image_size // 16), 1),
            nn.Sigmoid()  # Binary classification (real/fake)
        )

    def generate_noise(self, batch_size):
        return torch.randn(batch_size, self.latent_dim).to(next(self.parameters()).device)

    def forward(self, x, mode="generator"):
        if mode == "generator":
            return self.generator(x)
        elif mode == "discriminator":
            return self.discriminator(x)
        else:
            raise ValueError("Mode must be 'generator' or 'discriminator'.")


# Initialize model, optimizers, and loss function
def initialize_gan(image_size=128, latent_dim=100, lr=0.0002):
    model = GAN(image_size, latent_dim).to(device)
    optimizer_g = optim.Adam(model.generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(model.discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    loss_fn = nn.BCELoss()
    
    return model, optimizer_g, optimizer_d, loss_fn


In [3]:
# Functions to calculate unusual pixel mask and ratio (tericho spots in image)


def extract_color_mask(image, color_lower, color_upper):
    image_array = image.permute(1, 2, 0).numpy()  # Convert tensor to numpy array with shape (H, W, C)
    mask = np.all((image_array >= color_lower) & (image_array <= color_upper), axis=-1)
    color_ratio = np.sum(mask) / mask.size
    return mask, color_ratio

def calculate_color_distance(image):
    # Calculate mean values for each channel
    mean_r = torch.mean(image[0, :, :])
    mean_g = torch.mean(image[1, :, :])
    mean_b = torch.mean(image[2, :, :])

    # Calculate distances for each channel
    distances_r = torch.sqrt((image[0, :, :] - mean_r) ** 2)
    distances_g = torch.sqrt((image[1, :, :] - mean_g) ** 2)
    distances_b = torch.sqrt((image[2, :, :] - mean_b) ** 2)

    # Store distances in a dictionary
    distances = {
        'r': distances_r,
        'g': distances_g,
        'b': distances_b
    }

    return distances

def find_unusual_color(image):
    # Calculate distances
    distances = calculate_color_distance(image)

    # Compute the total distance for each pixel
    total_distance = distances['r'] + distances['g'] + distances['b']

    # Find the maximum total distance
    max_distance = torch.max(total_distance)

    # Get all indices where the total distance equals the max distance
    max_distance_indices = torch.nonzero(total_distance == max_distance, as_tuple=True)

    # Use the first occurrence of the maximum distance
    first_index = 0  # Change this to handle multiple matches differently if needed
    unusual_r = image[0, max_distance_indices[0][first_index], max_distance_indices[1][first_index]]
    unusual_g = image[1, max_distance_indices[0][first_index], max_distance_indices[1][first_index]]
    unusual_b = image[2, max_distance_indices[0][first_index], max_distance_indices[1][first_index]]

    return {
        'r': unusual_r.item(),
        'g': unusual_g.item(),
        'b': unusual_b.item(),
        'max_distance': max_distance.item()
    }

In [4]:

class TrainDataset(Dataset):
    def __init__(self, images_root, image_files, transform=None, augmentations=None):
        self.image_files = image_files
        self.images_root = images_root
        self.transform = transform
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(os.path.join(self.images_root, img_path)).convert('RGB')

        if self.augmentations:
            image = self.augmentations(image)
        
        if self.transform:
            image = self.transform(image)
            

        unusual_color = find_unusual_color(image)
        color_lower = np.array([(unusual_color['r'] - unusual_color['r']/10), (unusual_color['g'] - unusual_color['g']/2) ,(unusual_color['b'] - unusual_color['b']/10)])
        color_upper = np.array([(unusual_color['r'] + unusual_color['r']/2), (unusual_color['g'] + unusual_color['g']/5), (unusual_color['b'] + unusual_color['b']/2)])

        color_ratio = self._calculate_color_ratio(image , color_lower , color_upper)
        

        
        return image, torch.tensor(color_ratio, dtype=torch.float32)
    
    def _calculate_color_ratio(self, image , lower , upper):
        _, color_ratio = extract_color_mask(image , lower , upper)
        return color_ratio
    
class TestDataset(Dataset):
    def __init__(self, images_root, image_files, transform=None, augmentations=None):
        self.image_files = image_files
        self.images_root = images_root
        self.transform = transform
        self.augmentations = augmentations


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(os.path.join(self.images_root, img_path)).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        # Determine the label based on the filename or directory structure
        if 'tericho' in self.image_files[idx].lower():  # assuming 'terichomonas gallinae' indicates terichomonas gallinae-cropped images
            label = 1  # terichomonas gallinae present
        else:
            label = 0  # no terichomonas gallinae
        
        unusual_color = find_unusual_color(image)
        color_lower = np.array([(unusual_color['r'] - unusual_color['r']/10), 
                                (unusual_color['g'] - unusual_color['g']/2), 
                                (unusual_color['b'] - unusual_color['b']/10)])
        color_upper = np.array([(unusual_color['r'] + unusual_color['r']/2), 
                                (unusual_color['g'] + unusual_color['g']/5), 
                                (unusual_color['b'] + unusual_color['b']/2)])

        color_ratio = self._calculate_color_ratio(image , color_lower , color_upper)
        
        return image, torch.tensor(color_ratio, dtype=torch.float32), label
    
    def _calculate_color_ratio(self, image, lower, upper):
        _, color_ratio = extract_color_mask(image, lower, upper)
        return color_ratio

In [5]:
# vae_loss functions
def vae_loss(reconstructed, original, mu, logvar):
    """
    Compute the VAE loss.
      - Reconstruction loss: binary cross-entropy between reconstructed and original image.
      - KL divergence loss: forcing the latent space distribution closer to N(0,1)
    Returns a tuple of (total_loss, recon_loss, kl_loss)
    """
    bce = nn.functional.binary_cross_entropy(reconstructed, original, reduction='mean')
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    total_loss = bce + kl
    return total_loss, bce, kl

def vae_loss_inference(reconstructed, original, mu, logvar):
    recon_loss = nn.functional.mse_loss(reconstructed, original, reduction="none")
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_div



In [6]:
# Hyperparameters
input_dim = 224
latent_dim = 16
batch_size = 16
epochs = 150
early_stop_patience = 150
adv_loss_weight = 0.1
kl_loss_weight = 1.0
lr = 0.0001
early_stop_patient = 150
purple_lower = np.array([100, 0, 100])
purple_upper = np.array([255, 100, 255])

# Data loading
data_root = "./k-fold/fold_1"
splits = ["train", "eval", "test"]

data_dict = {}
datasets_dict = {}


data_transforms = transforms.Compose([
    RandomRotation(degrees=30),  # Randomly rotate images by up to 30 degrees
    RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally with a probability of 0.5
    RandomVerticalFlip(p=0.5),  # Randomly flip images vertically with a probability of 0.5
    RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0))  # Randomly crop and resize images to 128x128 with a scale between 80% and 100% of the original size
])


# Load CSV files and create datasets
for split in splits:
    try:
        # Load CSV data
        csv_path =  f"{data_root}/{split}/{split}.csv"
        data_dict[split] = pd.read_csv(csv_path)
        # Create dataset for each split
        datasets_dict[split] = TestDataset(
            images_root="./data/",
            image_files=data_dict[split]['filename'].values,
            transform=transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ]),
            augmentations=data_transforms if split == "train" else None
        )
    except Exception as e:
        print(f"Error loading {split} split: {e}")

# Create dataloaders
dataloaders = {
    split: DataLoader(
        datasets_dict[split],
        batch_size=batch_size if split == "train" else 1,
        shuffle=(split == "train"),
        num_workers=4,
        pin_memory=True
    ) for split in splits
}

print("Available splits:", list(datasets_dict.keys()))
print("Dataset sizes:", {split: len(dataset) for split, dataset in datasets_dict.items()})


Available splits: ['train', 'eval', 'test']
Dataset sizes: {'train': 122, 'eval': 16, 'test': 35}


In [8]:
import torch.nn.functional as F

def train_gan(model, dataloader, optimizer_g, optimizer_d, loss_fn, device, epochs=100):
    model.train()
    
    for epoch in range(epochs):
        total_d_loss, total_g_loss = 0, 0

        for real_images, _, _ in dataloader:
            batch_size = real_images.size(0)
            real_images = real_images.to(device)

            # Generate noise and fake images
            noise = model.generate_noise(batch_size)
            fake_images = model(noise, mode="generator")
            
            # Create real and fake labels
            real_labels = torch.ones(batch_size, 1, device=device)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            
            # ---------------------
            # Train Discriminator
            # ---------------------
            optimizer_d.zero_grad()
            
            real_preds = model(real_images, mode="discriminator")
            fake_preds = model(fake_images.detach(), mode="discriminator")
            
            real_loss = loss_fn(real_preds, real_labels)
            fake_loss = loss_fn(fake_preds, fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()
            
            total_d_loss += d_loss.item()
            
            # -----------------
            # Train Generator
            # -----------------
            optimizer_g.zero_grad()
            
            fake_preds = model(fake_images, mode="discriminator")
            g_loss = loss_fn(fake_preds, real_labels)  # Flip labels to trick discriminator
            g_loss.backward()
            optimizer_g.step()
            
            total_g_loss += g_loss.item()
        
        # Print losses
        print(f"Epoch {epoch+1}/{epochs} - D Loss: {total_d_loss/len(dataloader):.4f} - G Loss: {total_g_loss/len(dataloader):.4f}")

        # Save checkpoint every 10 epochs
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f'./models/gan_epoch_{epoch+1}.pth')
            print(f"Model checkpoint saved at epoch {epoch+1}")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, optimizer_g, optimizer_d, loss_fn = initialize_gan(image_size=224, latent_dim=100, lr=0.0002)

train_gan(model, dataloader=dataloaders['train'], optimizer_g=optimizer_g, optimizer_d=optimizer_d, loss_fn=loss_fn, device=device, epochs=100)


Epoch 1/100 - D Loss: 0.2520 - G Loss: 6.6606
Epoch 2/100 - D Loss: 0.0293 - G Loss: 12.1662
Epoch 3/100 - D Loss: 0.0054 - G Loss: 11.1088
Epoch 4/100 - D Loss: 0.0051 - G Loss: 9.6211
Epoch 5/100 - D Loss: 0.0061 - G Loss: 8.6352
Epoch 6/100 - D Loss: 0.0829 - G Loss: 37.0907
Epoch 7/100 - D Loss: 0.0000 - G Loss: 31.0876
Epoch 8/100 - D Loss: 0.7000 - G Loss: 42.6224
Epoch 9/100 - D Loss: 0.0000 - G Loss: 53.2782
Epoch 10/100 - D Loss: 0.0000 - G Loss: 53.1629
Model checkpoint saved at epoch 10
Epoch 11/100 - D Loss: 0.0000 - G Loss: 53.1300
Epoch 12/100 - D Loss: 0.0000 - G Loss: 53.1965
Epoch 13/100 - D Loss: 0.0000 - G Loss: 53.1783
Epoch 14/100 - D Loss: 0.0000 - G Loss: 53.1825
Epoch 15/100 - D Loss: 0.0000 - G Loss: 53.1326
Epoch 16/100 - D Loss: 0.0000 - G Loss: 53.1361
Epoch 17/100 - D Loss: 0.0000 - G Loss: 53.1584
Epoch 18/100 - D Loss: 0.0000 - G Loss: 53.1439
Epoch 19/100 - D Loss: 0.0000 - G Loss: 53.1791
Epoch 20/100 - D Loss: 0.0000 - G Loss: 53.1466
Model checkpoint 

In [9]:
# Save the trained model
# torch.save(model.state_dict(), "./models/vaae_model.pth")


In [16]:
def visualize_images(dataloaders, model, device, output_dir='reconstruction_images'):
    """
    Visualizes and saves a side-by-side comparison of real and generated images
    from the GAN model.

    Parameters:
        dataloaders (dict): A dictionary containing a 'test' DataLoader.
        model (nn.Module): The GAN model, expected to have a 'generator' attribute.
        device (torch.device): The device to run inference on.
        output_dir (str): Directory where the output images will be saved.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Set the model to evaluation mode
    model.eval()
    
    # Get a batch of real images from the test dataset.
    # We assume the first element of the batch is the images.
    test_iter = iter(dataloaders['test'])
    batch = next(test_iter)
    real_batch = batch[0].to(device)
    batch_size = real_batch.size(0)
    
    # Define the noise dimension (adjust if needed)
    noise_dim = 100
    # Generate random noise for each image in the batch
    noise = torch.randn(batch_size, noise_dim, device=device)
    
    with torch.no_grad():
        # Generate fake images using the generator
        fake_images = model.generator(noise)
    
    # Number of images to display (e.g., the first 4 images)
    num_to_display = min(4, batch_size)
    
    for i in range(num_to_display):
        # Get the i-th real and fake image and convert from CHW to HWC for display
        real_img = real_batch[i].cpu().permute(1, 2, 0).numpy()
        fake_img = fake_images[i].cpu().permute(1, 2, 0).numpy()
        
        # If pixel values are in the range [-1, 1], rescale them to [0, 1]
        if real_img.min() < 0:
            real_img = (real_img + 1) / 2
        if fake_img.min() < 0:
            fake_img = (fake_img + 1) / 2
        
        # Create a figure with two subplots: one for the real image and one for the fake image.
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        axes[0].imshow(real_img)
        axes[0].set_title("Real")
        axes[0].axis('off')
        axes[1].imshow(fake_img)
        axes[1].set_title("Fake")
        axes[1].axis('off')
        
        plt.tight_layout()
        
        # Save the figure to the output directory
        save_path = os.path.join(output_dir, f'image_{i}.png')
        plt.savefig(save_path)
        plt.close(fig)
        print(f"Saved: {save_path}")

# Example usage:
# Assuming you have defined 'dataloaders', 'model', and 'device' elsewhere:
visualize_images(dataloaders, model=model, device=device)


Saved: reconstruction_images/image_0.png


In [12]:
def detect_anomaly_threshold (vae, test_data_loader, device):  
    vae.eval()  
    reconstruction_losses = torch.empty(0, device=device)

    with torch.no_grad():
        for images, color_ratios, _ in test_data_loader:
            images, color_ratios = images.to(device), color_ratios.to(device)
            reconstructed, mu, logvar = vae(images, color_ratios)
            loss, _, _ = vae_loss(reconstructed, images, mu, logvar)
            print(loss)
            reconstruction_losses = torch.cat((reconstruction_losses, loss.unsqueeze(0)))
    
    # Compute threshold (test with just mean as a starting point)
    mean_loss = reconstruction_losses.mean()
    std_loss = reconstruction_losses.std()

    # Log the reconstruction loss distribution  
    # print(f"Reconstruction Losses: {reconstruction_losses[:10]}")  # Display first 10 losses  
    print(f"Mean Loss: {mean_loss}, Std: {std_loss}")

    anomaly_threshold = mean_loss + std_loss/2  # Using mean + std as a quick test  

    print(f"Anomaly Threshold: {anomaly_threshold}")  # Log the threshold  
    return anomaly_threshold  


In [13]:
def test_vae(vae, test_data_loader , anomaly_threshold, device):  
    vae.eval()  # Set the model to evaluation mode  
    anomalies = []
    true_labels = []  # True labels for the current batch  
    pred_labels = []
    with torch.no_grad():
        for images, color_ratios , label in test_data_loader:
            images, color_ratios = images.to(device), color_ratios.to(device)
            reconstructed, mu, logvar = vae(images, color_ratios)
            true_labels.extend(label)  # Store true labels (0-9)
            loss = vae_loss_inference(reconstructed, images, mu, logvar)
            vaeloss,_,_ = vae_loss(reconstructed, images, mu, logvar)

            print(vaeloss)
            mean_loss = loss.view(loss.size(0), -1).mean(dim=1)
            print(mean_loss)
            for i, reconstruction_loss in enumerate(mean_loss):
                if reconstruction_loss.item() > anomaly_threshold:
                    anomalies.append(( "no tericho", reconstruction_loss.item()))
                    pred_labels.append(0)
                else:
                    anomalies.append(("tericho", reconstruction_loss.item()))
                    pred_labels.append(1)

    return images, reconstructed ,anomalies, true_labels, pred_labels   # Return original and reconstructed images  


In [15]:
th = detect_anomaly_threshold(vae=model,test_data_loader=dataloaders['eval'],device=device)
print(th)

tensor(218.1273, device='cuda:0')
tensor(237.7959, device='cuda:0')
tensor(236.1777, device='cuda:0')
tensor(239.1119, device='cuda:0')
tensor(246.4505, device='cuda:0')
tensor(261.5175, device='cuda:0')
tensor(254.6542, device='cuda:0')
tensor(248.2431, device='cuda:0')
tensor(211.4664, device='cuda:0')
tensor(214.5368, device='cuda:0')
tensor(219.7766, device='cuda:0')
tensor(219.4080, device='cuda:0')
tensor(217.6664, device='cuda:0')
tensor(219.8562, device='cuda:0')
tensor(213.7203, device='cuda:0')
tensor(211.4275, device='cuda:0')
Mean Loss: 229.3710174560547, Std: 16.704294204711914
Anomaly Threshold: 237.72315979003906
tensor(237.7232, device='cuda:0')


In [16]:
images, reconstructed ,anomalies, true_labels, pred_labels = test_vae(model , dataloaders['test'] , th ,device)

tensor(221.3050, device='cuda:0')
tensor([221.3050], device='cuda:0')
tensor(157.3258, device='cuda:0')
tensor([157.3258], device='cuda:0')
tensor(162.9765, device='cuda:0')
tensor([162.9765], device='cuda:0')
tensor(174.2538, device='cuda:0')
tensor([174.2538], device='cuda:0')
tensor(171.0129, device='cuda:0')
tensor([171.0129], device='cuda:0')
tensor(173.7413, device='cuda:0')
tensor([173.7413], device='cuda:0')
tensor(171.8921, device='cuda:0')
tensor([171.8921], device='cuda:0')
tensor(204.6294, device='cuda:0')
tensor([204.6294], device='cuda:0')
tensor(158.1421, device='cuda:0')
tensor([158.1421], device='cuda:0')
tensor(178.7417, device='cuda:0')
tensor([178.7417], device='cuda:0')
tensor(207.5408, device='cuda:0')
tensor([207.5408], device='cuda:0')
tensor(196.4154, device='cuda:0')
tensor([196.4154], device='cuda:0')
tensor(223.1369, device='cuda:0')
tensor([223.1368], device='cuda:0')
tensor(247.0985, device='cuda:0')
tensor([247.0985], device='cuda:0')
tensor(217.1849, dev

In [17]:
from sklearn.metrics import classification_report, confusion_matrix , f1_score
f1 = f1_score(true_labels, pred_labels)
print(f1)
# print(classification_report(true_labels, pred_labels))
# print(confusion_matrix(true_labels, pred_labels))

0.8709677419354839


In [18]:
print(pred_labels)
print(true_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
[tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1)]


In [19]:
print(anomalies[:8])

[('wormed', 221.3050079345703), ('wormed', 157.3258056640625), ('wormed', 162.9765167236328), ('wormed', 174.2538299560547), ('wormed', 171.01290893554688), ('wormed', 173.7413330078125), ('wormed', 171.89207458496094), ('wormed', 204.62937927246094)]


In [20]:
print(anomalies[8:])


[('wormed', 158.14205932617188), ('wormed', 178.74172973632812), ('wormed', 207.54083251953125), ('wormed', 196.41542053222656), ('wormed', 223.1368408203125), ('no wormed', 247.09848022460938), ('wormed', 217.18492126464844), ('wormed', 192.02735900878906), ('wormed', 186.51715087890625), ('wormed', 219.11318969726562), ('wormed', 223.01605224609375), ('no wormed', 253.3943634033203), ('wormed', 230.77816772460938), ('wormed', 207.63681030273438), ('no wormed', 343.3358154296875), ('wormed', 187.82064819335938), ('no wormed', 387.0472106933594), ('no wormed', 355.0072021484375), ('no wormed', 351.0190124511719), ('no wormed', 369.83599853515625), ('wormed', 193.2080535888672), ('wormed', 210.8118438720703), ('no wormed', 282.4904479980469), ('wormed', 227.77255249023438), ('wormed', 198.7176055908203), ('wormed', 193.2436981201172), ('wormed', 219.24148559570312)]
